In [ ]:
player_salaries = []
for idx in range(1, len(tables), 1):  # player tables at odd indices
    df = pd.read_html(str(table))[0]
    player_salaries.append(df)

df_players = pd.concat(player_salaries, ignore_index=True)
print(df_players[['Player', 'Squad']].head(10))


In [ ]:
# Example cleaning for GBP or USD
import re
def extract_usd_amount(wage_str):
    match = re.search(r'\$([\d,]+)', str(wage_str))
    if match:
        return float(match.group(1).replace(',', ''))
    return None

df_players['Annual_Wages_USD'] = df_players['Annual Wages'].astype(str).apply(extract_usd_amount)
df_players['Salary_M_USD'] = df_players['Annual_Wages_USD'] / 1_000_000

# Standardize names
df_players['Player'] = df_players['Player'].str.lower().str.strip()
df_players['Squad'] = df_players['Squad'].str.lower().str.strip()
df_all_conversion['Player'] = df_all_conversion['Player'].str.lower().str.strip()
df_all_conversion['Squad'] = df_all_conversion['Squad'].str.lower().str.strip()

# Merge!
df_stats_salary = pd.merge(
    df_all_conversion,
    df_players[['Player', 'Squad', 'Salary_M_USD']],
    on=['Player', 'Squad'],
    how='left'
)


In [ ]:
import re
import pandas as pd

# Extract USD salary (if you haven't already)
def extract_usd_amount(wage_str):
    match = re.search(r'\$([\d,]+)', str(wage_str))
    if match:
        return float(match.group(1).replace(',', ''))
    return None

df_players['Annual_Wages_USD'] = df_players['Annual Wages'].astype(str).apply(extract_usd_amount)
df_players['Salary_M_USD'] = df_players['Annual_Wages_USD'] / 1_000_000

# Standardize for merge
df_players['Player'] = df_players['Player'].str.lower().str.strip()
df_players['Squad'] = df_players['Squad'].str.lower().str.strip()
df_all_conversion['Player'] = df_all_conversion['Player'].str.lower().str.strip()
df_all_conversion['Squad'] = df_all_conversion['Squad'].str.lower().str.strip()


In [ ]:
import unicodedata

def remove_accents(text):
    if pd.isnull(text):
        return ""
    return ''.join(
        c for c in unicodedata.normalize('NFKD', str(text))
        if not unicodedata.combining(c)
    )

df_players['Player'] = df_players['Player'].apply(remove_accents).str.lower().str.strip()
df_players['Squad'] = df_players['Squad'].apply(remove_accents).str.lower().str.strip()

# Do the same for your stats DataFrame:
df_all_conversion['Player'] = df_all_conversion['Player'].apply(remove_accents).str.lower().str.strip()
df_all_conversion['Squad'] = df_all_conversion['Squad'].apply(remove_accents).str.lower().str.strip()


In [ ]:
df_stats_salary = pd.merge(
    df_all_conversion,
    df_players[['Player', 'Squad', 'Salary_M_USD']],
    on=['Player', 'Squad'],
    how='left'
)


In [ ]:
# Make sure these columns are numeric
df_stats_salary['Goals'] = pd.to_numeric(df_stats_salary['Goals'], errors='coerce')
df_stats_salary['xG'] = pd.to_numeric(df_stats_salary['xG'], errors='coerce')
df_stats_salary['Salary_M_USD'] = pd.to_numeric(df_stats_salary['Salary_M_USD'], errors='coerce')

# Filter for players with salary data and at least 1 goal or xG
df_stats_salary = df_stats_salary[df_stats_salary['Salary_M_USD'] > 0]
df_stats_salary = df_stats_salary[df_stats_salary['Goals'].notnull() | df_stats_salary['xG'].notnull()]

# ROI and value metrics
df_stats_salary['Goals_per_Million'] = df_stats_salary['Goals'] / df_stats_salary['Salary_M_USD']
df_stats_salary['xG_per_Million'] = df_stats_salary['xG'] / df_stats_salary['Salary_M_USD']
df_stats_salary['G_minus_xG'] = df_stats_salary['Goals'] - df_stats_salary['xG']


In [ ]:
print("Top ROI (Goals per $1M):")
print(
    df_stats_salary
    .sort_values('Goals_per_Million', ascending=False)
    [['Player', 'Squad', 'Goals', 'xG', 'Salary_M_USD', 'Goals_per_Million', 'G_minus_xG']]
    .head(15)
)

print("\nTop Overperformers (G-xG):")
print(
    df_stats_salary
    .sort_values('G_minus_xG', ascending=False)
    [['Player', 'Squad', 'Goals', 'xG', 'Salary_M_USD', 'G_minus_xG']]
    .head(10)
)


In [ ]:
# Lowercase and strip for matching
import unicodedata

def remove_accents(text):
    if pd.isnull(text):
        return ""
    return ''.join(
        c for c in unicodedata.normalize('NFKD', str(text))
        if not unicodedata.combining(c)
    )

for col in ['Player', 'Squad']:
    df_all_conversion[col] = df_all_conversion[col].apply(remove_accents).str.lower().str.strip()
    df_players[col] = df_players[col].apply(remove_accents).str.lower().str.strip()

df_all_conversion['xG'] = pd.to_numeric(df_all_conversion['Expected_xG'], errors='coerce')
df_all_conversion['G_minus_xG'] = df_all_conversion['Goals'] - df_all_conversion['xG']
df_all_conversion['xG_per_shot'] = df_all_conversion['xG'] / df_all_conversion['Shots']

# Merge on player+club
df_all_value = pd.merge(
    df_all_conversion,
    df_players[['Player', 'Squad', 'Salary_M_USD']],
    on=['Player', 'Squad'],
    how='left'
)


In [ ]:
# Save ALL columns of df_all_value
filename = "player_team_salary_stats_2024-2025_SerieA.csv"
df_all_value.to_csv(filename, index=False)
print(f"Saved all player/team/salary/stats/value data to: {filename}")
